# D3 HealthCare Data
Made by -
- Akshat Lakhara
- Anil Kumar Dhayal
- Deepshikha Kumari
- Disha Kamlesh Kumar Shah
- Swaraj Haresh Purohit
- Vidhi Sachin Thakare 

In [3]:
!pip install numpy

!pip install rich
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install scipy.stats
!pip install scikit-learn

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement rich
ERROR: No matching distribution found for rich
DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will rem

# Install and Import all the dependencies

In [4]:

import findspark
findspark.init('/usr/local/spark')

import pyspark

import warnings

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("peer review").getOrCreate()

warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency
from rich import print

## Merging datasets to create a consolidated datafame
Reading data and using join method to create a consolidated datafame named ```merged_data```

In [5]:
filename="csv/Liver Patient Dataset.csv"
data= pd.read_csv(filename, encoding='unicode_escape')
data.set_index("ID")

,Age of the patient,Gender of the patient,Total Bilirubin,Direct Bilirubin,Alkphos Alkaline Phosphotase,Sgpt Alamine Aminotransferase,Sgot Aspartate Aminotransferase,Total Protiens,ALB Albumin,A/G Ratio Albumin and Globulin Ratio,City_Code,State_Code
ID,,,,,,,,,,,,
ID_001,65.0,Female,0.7,0.1,187.0,16.0,18.0,6.8,3.3,0.90,CT01,ST15
ID_002,62.0,Male,10.9,5.5,699.0,64.0,100.0,7.5,3.2,0.74,CT28,ST11
ID_003,62.0,Male,7.3,4.1,490.0,60.0,68.0,7.0,3.3,0.89,CT12,ST28
ID_004,58.0,Male,1.0,0.4,182.0,14.0,20.0,6.8,3.4,1.00,CT21,ST27
ID_005,72.0,Male,3.9,2.0,195.0,27.0,59.0,7.3,2.4,0.40,CT03,ST24
ID_006,46.0,Male,1.8,0.7,208.0,19.0,14.0,7.6,4.4,1.30,CT06,ST22
ID_007,26.0,Female,0.9,0.2,154.0,NaN,12.0,7.0,3.5,1.00,CT06,ST22
ID_008,29.0,Female,0.9,0.3,202.0,14.0,11.0,6.7,3.6,1.10,CT02,ST15
ID_009,17.0,Male,0.9,0.3,202.0,22.0,19.0,7.4,4.1,1.20,CT26,ST15


In [6]:
state= pd.read_csv("csv/Statecode_name.csv")
status= pd.read_csv("csv/Status.csv")
city= pd.read_csv("csv/Cityname_statecode.csv")

In [7]:
state.rename(columns={"State Code":"State_Code"},inplace=True)
state=state.set_index("State_Code")
city.rename(columns={"City Code":"City_Code"},inplace=True)
city=city.set_index("City_Code")
status.replace(2, 0, inplace=True)

In [8]:
merge_data=pd.merge(data, state, on='State_Code', how='left')
merge_data=pd.merge(merge_data, city[["City name"]], on='City_Code', how='left')
merge_data=pd.merge(merge_data, status, on='ID', how='outer')

In [9]:
merge_data=merge_data.set_index("ID")
merge_data.loc[merge_data["Gender of the patient"]=="Female","Gender of the patient"]=1 
merge_data.loc[merge_data["Gender of the patient"]=="Male","Gender of the patient"]=0 

In [10]:
merge_data=spark.createDataFrame(merge_data)

TypeError: Can not merge type <class 'pyspark.sql.types.LongType'> and <class 'pyspark.sql.types.DoubleType'>

# Is the dataset balanced?

In [ ]:
fig = plt.figure(figsize=(5,4))
plt.pie(merge_data['Status'].value_counts(),labels=['Liver patient','Non-Liver patient'],radius=1.2,
colors = ['black','green'],autopct='%1.2f%%',pctdistance=1.5 ,labeldistance=1.15,startangle =70)

plt.legend(title = 'Dataset',loc='upper right', bbox_to_anchor=(1.3,1.3))
plt.show()

In [ ]:
numb_col=['Age of the patient', 'Gender of the patient', 'Total Bilirubin',
       'Direct Bilirubin', 'Alkphos Alkaline Phosphotase',
       'Sgpt Alamine Aminotransferase', 'Sgot Aspartate Aminotransferase',
       'Total Protiens', 'ALB Albumin', 'A/G Ratio Albumin and Globulin Ratio','Status']

In [ ]:
plt.figure(figsize=(16,14))
for i,col in enumerate(merge_data[numb_col].columns):
    plt.subplot(4,3,i+1)
    sns.distplot(a=merge_data[col],bins=50,hist=True)
    plt.tight_layout()

form the above plot we can infer that there are a lot of discrapnacies in the data and is not balanced

In [11]:
target_distribution = merge_data['Status'].value_counts()

# Check for missing values and duplicates
is_dataset_balanced = target_distribution.min() / target_distribution.max() > 0.8

print("Target Distribution: where 1 indicates Liver Patient, 0 indicates not Liver Patient")
print(target_distribution)

target_distribution = merge_data['Gender of the patient'].value_counts()

print("Target Distribution: where 1 indicates Female Patient, 0 indicates not Male Patient")
print(target_distribution)


# Check for missing values and duplicates
is_dataset_balanced = target_distribution.min() / target_distribution.max() > 0.8


print("\nIs the dataset balanced?", is_dataset_balanced)


Target Distribution: where 1 indicates Liver Patient, 0 indicates not Liver Patient

Status
1    21907
0     8770
Name: count, dtype: int64

Target Distribution: where 1 indicates Female Patient, 0 indicates not Male Patient

Gender of the patient
0    21977
1     7802
Name: count, dtype: int64

Is the dataset balanced? False

In [ ]:
correlation_matrix = merge_data.corr(method='pearson',numeric_only=True)
liver_d_corr = correlation_matrix["Status"].abs().sort_values(ascending=False)

print(liver_d_corr)

plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix,annot=True , cmap="coolwarm",fmt=".2f",linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

from this we can observe that ```Total Bilirubin``` and ```Total Bilirubin``` are highly correlated to each other and are the most dominat corr with status

# Any missing values?

using ```.isnull()``` function we calculate the number of 

In [ ]:
missing_val=merge_data.isnull().sum()
missing_val[missing_val>0]

In [ ]:
has_missing_values = merge_data.isnull().values.any()
print("Any missing values?", has_missing_values)
merge_data.loc[merge_data.isnull().any(axis=1)==True]

we can observe that there are a lot of missing values and with different distribution it is to keep in mind that we can use the reltions which we found above


using a reggresion model we can get data from another rows to get data which is not avaiable

In [ ]:
missing_val=merge_data.isnull().sum()
missing_val[missing_val>0]

In [ ]:
# Fixing Missing Value
from sklearn.linear_model import LinearRegression
df_corr = merge_data.dropna(subset=["Total Bilirubin", "Direct Bilirubin"])

# Create input (x) and target (y) variables
x = df_corr[["Total Bilirubin"]]
y = df_corr["Direct Bilirubin"]

# Fit the linear regression model
model = LinearRegression()
model.fit(x, y)

def impute_direct_bilirubin(row):
    if not pd.isnull(row["Direct Bilirubin"]):
        return row["Direct Bilirubin"]
    elif pd.isnull(row["Direct Bilirubin"]) and not pd.isnull(row["Total Bilirubin"]):
        return model.predict([[row["Total Bilirubin"]]])[0]
    else:
        return row["Direct Bilirubin"]

merge_data["Direct Bilirubin"] = merge_data.apply(impute_direct_bilirubin, axis=1)

df_corr = merge_data.dropna(subset=["Total Bilirubin", "Direct Bilirubin"])

x_reverse = df_corr[["Direct Bilirubin"]]
y_reverse = df_corr["Total Bilirubin"]


model_reverse = LinearRegression()
model_reverse.fit(x_reverse, y_reverse)

def impute_total_bilirubin(row):
    if not pd.isnull(row["Total Bilirubin"]):
        return row["Total Bilirubin"]
    elif pd.isnull(row["Total Bilirubin"]) and not pd.isnull(row["Direct Bilirubin"]):
        return model_reverse.predict([[row["Direct Bilirubin"]]])[0]
    else:
        return row["Total Bilirubin"]

merge_data["Total Bilirubin"] = merge_data.apply(impute_total_bilirubin, axis=1)


filling na values with the distribution they belong to

In [ ]:
merge_data["Alkphos Alkaline Phosphotase"].fillna(merge_data["Alkphos Alkaline Phosphotase"].median(),inplace=True)
merge_data["Sgpt Alamine Aminotransferase"].fillna(merge_data["Sgpt Alamine Aminotransferase"].median(),inplace=True)
merge_data["Sgot Aspartate Aminotransferase"].fillna(merge_data["Sgot Aspartate Aminotransferase"].mean(),inplace=True)
merge_data["Total Protiens"].fillna(merge_data["Total Protiens"].mean(),inplace=True)
merge_data["ALB Albumin"].fillna(merge_data["ALB Albumin"].mean(),inplace=True)
merge_data["A/G Ratio Albumin and Globulin Ratio"].fillna(merge_data["A/G Ratio Albumin and Globulin Ratio"].mean(),inplace=True)
merge_data["Age of the patient"].fillna(merge_data["Age of the patient"].mean(),inplace=True)
merge_data["Gender of the patient"].fillna('not_mentioned',inplace=True)


In [ ]:
missing_val=merge_data.isnull().sum()
missing_val[missing_val>0]

In [ ]:
merge_clean_data=merge_data.dropna()

In [ ]:
merge_clean_data

# Are there duplicate records?

In [ ]:
has_duplicates = merge_data.duplicated().any()
print("Are there duplicate records?", has_duplicates)
#merge_data[merge_data.duplicated()==True]
merge_data.loc[merge_data.duplicated(keep='first')==True]

In [ ]:
merge_clean_data=merge_clean_data.drop_duplicates()

we also know that ```Total Bilirubin``` and ```Total Bilirubin``` are highly correlated to each other and are the most dominat we can drop any one of the col

In [ ]:
merge_clean_data.drop('Total Bilirubin', axis=1, inplace= True)

In [ ]:
merge_clean_data

# Any affect of age group on the occurrence of liver disease.

In [ ]:
correlation_matrix = merge_clean_data[["Age of the patient","Status"]].corr(method='pearson',numeric_only=True)
liver_d_corr_with_age = correlation_matrix["Status"].sort_values(ascending=False)
print(liver_d_corr_with_age)
print("\nAny affect of age group on the occurrence of liver disease?", liver_d_corr_with_age["Age of the patient"]>0.1)

In [ ]:
# Box plot of age distribution for each health status
plt.figure(figsize=(10, 6))
sns.boxplot(x='Status', y='Age of the patient', data=merge_clean_data, palette='Set2')
plt.title('Box Plot of Age for Health Status')
plt.xlabel('Health Status')
plt.ylabel('Age')
plt.show()

Also from the box plot we can infer as this indipendence becaouse the distribution follows a normal curve

# Any relationship between diff factor 

In [ ]:
correlation_matrix = merge_clean_data.corr(method='pearson',numeric_only=True)
liver_d_corr = correlation_matrix["Status"].abs().sort_values(ascending=False)

print(liver_d_corr)

plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix,annot=True , cmap="coolwarm",fmt=".2f",linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
correlation_matrix = merge_data[["Age of the patient","Status"]].corr(method='pearson',numeric_only=True)
liver_d_corr_with_age = correlation_matrix["Status"].sort_values(ascending=False)
print(liver_d_corr_with_age)
print("\nAny affect of age group on the occurrence of liver disease?", liver_d_corr_with_age["Age of the patient"]>0.1)

## is there any corelation between Gender of the patient and presence of liver disease

In [ ]:
CrosstabResult=pd.crosstab(index=merge_clean_data['Gender of the patient'],columns=merge_data['Status'])
print(CrosstabResult) 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value < 0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])
print(ChiSqResult[1]<0.05)

## is there any corelation between state of the patient and presence of liver disease

In [ ]:
CrosstabResult=pd.crosstab(index=merge_clean_data['State Name'],columns=merge_data['Status'])
print(CrosstabResult) 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value < 0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])
print(ChiSqResult[1]<0.05)